In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
dataset_path = '/content/drive/MyDrive/EDGE AI/df'
os.listdir(dataset_path)

['cracks', 'decoloration', 'dust', 'organic']

## Installing the necessary dependencies

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.utils as vutils
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Transformation of the input images

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # simple augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

dataset = ImageFolder(root="/content/drive/MyDrive/EDGE AI/df", transform=transform)

## Weighting the labels

In [ ]:
import numpy as np
from collections import Counter

targets = [label for _, label in dataset]
class_counts = Counter(targets)
print("Class counts:", class_counts)

# Inverse frequency for weights (higher weight for minority class)
weights = [1.0 / class_counts[i] for i in range(len(class_counts))]
class_weights = torch.FloatTensor(weights).to(device)

total_samples = sum(class_counts.values())
weight_cracks = total_samples / class_counts[0]
weight_decoloration = total_samples / class_counts[1]
weight_dust = total_samples / class_counts[2]
weight_organic = total_samples / class_counts[3]

Class counts: Counter({3: 865, 2: 862, 1: 501, 0: 312})


In [ ]:
print("Class Weights Tensor:", class_weights)
print(f"Weight Cracks: {weight_cracks}")
print(f"Weight Decoloration: {weight_decoloration}")
print(f"Weight Dust: {weight_dust}")
print(f"Weight Organic: {weight_organic}")

Class Weights Tensor: tensor([0.0032, 0.0020, 0.0012, 0.0012], device='cuda:0')
Weight Cracks: 8.14102564102564
Weight Decoloration: 5.069860279441118
Weight Dust: 2.9466357308584685
Weight Organic: 2.9364161849710984


In [ ]:
# Split into train/val (80/20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## AlexNet

In [ ]:
class AlexNet(nn.Module):
  def __init__(self, num_classes = 2):
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size = 11, stride = 4, padding = 2),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = 3, stride = 2),

        nn.Conv2d(64, 192, kernel_size = 5, padding = 2),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = 3, stride = 2),

        nn.Conv2d(192, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),

        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),

        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256 * 6 * 6, 4096),
        nn.ReLU(inplace=True),

        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),

        nn.Linear(4096, num_classes)
    )

  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

In [ ]:
model_alex = AlexNet(num_classes=2)
model_alex = model_alex.to(device)

In [ ]:
optimizer = optim.Adam(model_alex.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

In [ ]:
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
import time

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
  start_time = time.time()

  model_alex.train()
  train_loss = 0.0
  correct = 0
  total = 0

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model_alex(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  train_accuracy = 100 * correct / total
  avg_train_loss = train_loss / len(train_loader)

  model_alex.eval()
  val_loss = 0.0
  correct = 0
  total = 0

  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model_alex(images)
      loss = criterion(outputs, labels)

      val_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  val_accuracy = 100 * correct / total
  avg_val_loss = val_loss / len(val_loader)
  end_time = time.time()

  print(f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
        f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}% | "
        f"Time: {time.time() - start_time:.2f}s")

In [ ]:
torch.save(model_alex.state_dict(), "model_alex.pth")
print("AlexNet model saved successfully.")

In [ ]:
# model_alex = AlexNet(num_classes=2)
# model_alex.load_state_dict(torch.load("model_alex.pth"))
# model_alex = model_alex.to(device)
# model_alex.eval()

## Custom CNN

In [ ]:
class CustomCNN(nn.Module):
  def __init__(self, num_classes = 2):
    super(CustomCNN, self).__init__()

    self.conv_layers = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(16, 32, kernel_size = 3, padding = 1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 14 * 14, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

  def forward(self, x):
      x = self.conv_layers(x)
      x = self.fc_layers(x)
      return x

In [ ]:
model_custom = CustomCNN(num_classes=2)
model_custom = model_custom.to(device)

In [ ]:
optimizer = torch.optim.Adam(model_custom.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [ ]:
outputs = model_custom(images)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    start_time = time.time()

    model_custom.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_custom(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    model_custom.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_custom(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader)
    end_time = time.time()

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}% | "
          f"Time: {end_time - start_time:.2f}s")

In [ ]:
# Save the model's state_dict (recommended way)
torch.save(model_custom.state_dict(), 'custom_cnn_model.pth')
print("Model saved successfully!")

## RESNET

In [ ]:
import time

In [ ]:
import torchvision.models as models

In [ ]:
model_resnet = models.resnet18(pretrained=False)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 2)
model_resnet = model_resnet.to(device)

In [ ]:
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=0.001)
class_weights = torch.tensor([weight_clean, weight_dust], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    start_time = time.time()

    model_resnet.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    model_resnet.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_resnet(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader)
    end_time = time.time()

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}% | "
          f"Time: {end_time - start_time:.2f}s")


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10] Train Loss: 0.6493, Train Acc: 70.02% | Val Loss: 0.6075, Val Acc: 71.18% | Time: 98.16s
Epoch [2/10] Train Loss: 0.5261, Train Acc: 75.25% | Val Loss: 0.7028, Val Acc: 67.50% | Time: 96.19s
Epoch [3/10] Train Loss: 0.5208, Train Acc: 75.64% | Val Loss: 0.5288, Val Acc: 76.60% | Time: 96.23s
Epoch [4/10] Train Loss: 0.5021, Train Acc: 77.43% | Val Loss: 0.5093, Val Acc: 79.88% | Time: 96.47s
Epoch [5/10] Train Loss: 0.4751, Train Acc: 79.85% | Val Loss: 0.4995, Val Acc: 78.72% | Time: 95.42s
Epoch [6/10] Train Loss: 0.4914, Train Acc: 78.55% | Val Loss: 0.5198, Val Acc: 76.79% | Time: 95.57s
Epoch [7/10] Train Loss: 0.4834, Train Acc: 79.08% | Val Loss: 0.5068, Val Acc: 79.11% | Time: 94.58s
Epoch [8/10] Train Loss: 0.4609, Train Acc: 79.47% | Val Loss: 0.4748, Val Acc: 76.98% | Time: 94.54s
Epoch [9/10] Train Loss: 0.4701, Train Acc: 78.79% | Val Loss: 0.4864, Val Acc: 76.40% | Time: 96.56s
Epoch [10/10] Train Loss: 0.4386, Train Acc: 80.97% | Val Loss: 0.4815, Val Acc: 7

In [ ]:
torch.save(model_resnet.state_dict(), "resnet_solar_model.pth")
print("Model saved as 'resnet18_solar_model.pth'")

Model saved as 'resnet18_solar_model.pth'


In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.residual(x)
        out += self.shortcut(x)
        return self.relu(out)

In [ ]:
class ResNet8(nn.Module):
    def __init__(self, num_classes=4):
        super(ResNet8, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1, stride=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.layer1 = BasicBlock(16, 32, stride=2)
        self.layer2 = BasicBlock(32, 64, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
model_resnet8 = ResNet8(num_classes=4).to(device)

In [ ]:
optimizer = optim.Adam(model_resnet8.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    start_time = time.time()

    model_resnet8.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_resnet8(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model_resnet8.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_resnet8(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}% | "
          f"Time: {time.time() - start_time:.2f}s")

Epoch [1/10] Train Loss: 0.6745, Train Acc: 73.38% | Val Loss: 0.6891, Val Acc: 75.20% | Time: 124.95s
Epoch [2/10] Train Loss: 0.4911, Train Acc: 81.69% | Val Loss: 0.7403, Val Acc: 68.31% | Time: 127.70s
Epoch [3/10] Train Loss: 0.4507, Train Acc: 83.12% | Val Loss: 0.3541, Val Acc: 87.60% | Time: 125.58s
Epoch [4/10] Train Loss: 0.3891, Train Acc: 84.94% | Val Loss: 0.3308, Val Acc: 87.80% | Time: 123.88s
Epoch [5/10] Train Loss: 0.3692, Train Acc: 86.32% | Val Loss: 0.2925, Val Acc: 89.96% | Time: 124.47s
Epoch [6/10] Train Loss: 0.3285, Train Acc: 88.24% | Val Loss: 0.5072, Val Acc: 77.76% | Time: 125.94s
Epoch [7/10] Train Loss: 0.3502, Train Acc: 87.50% | Val Loss: 0.9357, Val Acc: 60.83% | Time: 124.42s
Epoch [8/10] Train Loss: 0.2985, Train Acc: 88.04% | Val Loss: 0.2551, Val Acc: 92.13% | Time: 125.18s
Epoch [9/10] Train Loss: 0.2701, Train Acc: 90.85% | Val Loss: 0.2621, Val Acc: 91.93% | Time: 124.57s
Epoch [10/10] Train Loss: 0.2572, Train Acc: 91.24% | Val Loss: 0.4642, V

In [ ]:
torch.save(model_resnet8.state_dict(), "model_resnet8.pth")
print("ResNet8 model saved successfully.")

ResNet8 model saved successfully.


## RESNET 10

In [ ]:
class ResNet10(nn.Module):
    def __init__(self, num_classes=4):
        super(ResNet10, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1, stride=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.layer1 = nn.Sequential(
            BasicBlock(16, 32, stride=2),
            BasicBlock(32, 32)
        )
        self.layer2 = nn.Sequential(
            BasicBlock(32, 64, stride=2),
            BasicBlock(64, 64)
        )
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
model_resnet10 = ResNet10(num_classes=4).to(device)

In [ ]:
optimizer = optim.Adam(model_resnet10.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    start_time = time.time()

    # Training phase
    model_resnet10.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_resnet10(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    avg_train_loss = running_loss / len(train_loader)

    # Validation phase
    model_resnet10.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_resnet10(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader)

    end_time = time.time()

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}% | "
          f"Time: {end_time - start_time:.2f}s")

Epoch [1/10] Train Loss: 0.5850, Train Acc: 75.89% | Val Loss: 1.3813, Val Acc: 54.72% | Time: 123.87s
Epoch [2/10] Train Loss: 0.4325, Train Acc: 83.27% | Val Loss: 0.5451, Val Acc: 75.98% | Time: 124.38s
Epoch [3/10] Train Loss: 0.3856, Train Acc: 85.38% | Val Loss: 0.3303, Val Acc: 86.81% | Time: 124.37s
Epoch [4/10] Train Loss: 0.3435, Train Acc: 86.66% | Val Loss: 0.4399, Val Acc: 81.50% | Time: 123.79s
Epoch [5/10] Train Loss: 0.2771, Train Acc: 89.76% | Val Loss: 0.4390, Val Acc: 78.35% | Time: 124.96s
Epoch [6/10] Train Loss: 0.2842, Train Acc: 89.52% | Val Loss: 0.2620, Val Acc: 89.76% | Time: 125.49s
Epoch [7/10] Train Loss: 0.2591, Train Acc: 90.85% | Val Loss: 0.4455, Val Acc: 79.53% | Time: 124.28s
Epoch [8/10] Train Loss: 0.2224, Train Acc: 91.29% | Val Loss: 0.2639, Val Acc: 91.34% | Time: 124.33s
Epoch [9/10] Train Loss: 0.2568, Train Acc: 90.40% | Val Loss: 0.4944, Val Acc: 80.71% | Time: 124.98s
Epoch [10/10] Train Loss: 0.2400, Train Acc: 90.31% | Val Loss: 0.2496, V

In [ ]:
torch.save(model_resnet10.state_dict(), "model_resnet10.pth")
print("ResNet8 model saved successfully.")

ResNet8 model saved successfully.


## SqueezeNet

In [ ]:
model_squeezenet = models.squeezenet1_0(pretrained=False)
num_classes = 2
model_squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model_squeezenet.num_classes = num_classes
model_squeezenet = model_squeezenet.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_squeezenet.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model_squeezenet.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_squeezenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100.*correct/total:.2f}%')

In [ ]:
torch.save(model_squeezenet.state_dict(), 'model_squeezenet.pth')

## Efficient Net

In [ ]:
num_classes = 4

In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 846.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.5 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=4f571662f3847c08e6b7ccb0a7cb4fa87fb159a45f1134c

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
model_efficientnet = EfficientNet.from_name('efficientnet-b0')
model_efficientnet._fc = nn.Linear(model_efficientnet._fc.in_features, num_classes)
model_efficientnet = model_efficientnet.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_efficientnet.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    start_time = time.time()
    model_efficientnet.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_efficientnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    end_time = time.time()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100.*correct/total:.2f}%, Time: {end_time - start_time:.2f}s')

Epoch [1/10], Loss: 0.8450, Accuracy: 66.24%, Time: 120.05s
Epoch [2/10], Loss: 0.5428, Accuracy: 79.48%, Time: 108.99s
Epoch [3/10], Loss: 0.4552, Accuracy: 83.91%, Time: 109.65s
Epoch [4/10], Loss: 0.3481, Accuracy: 86.81%, Time: 109.99s
Epoch [5/10], Loss: 0.3261, Accuracy: 88.34%, Time: 110.52s
Epoch [6/10], Loss: 0.2480, Accuracy: 90.65%, Time: 110.65s
Epoch [7/10], Loss: 0.2340, Accuracy: 92.03%, Time: 109.94s
Epoch [8/10], Loss: 0.2468, Accuracy: 91.29%, Time: 110.12s
Epoch [9/10], Loss: 0.2017, Accuracy: 93.06%, Time: 110.52s
Epoch [10/10], Loss: 0.1991, Accuracy: 93.65%, Time: 111.34s


In [ ]:
torch.save(model_efficientnet.state_dict(), 'model_efficient.pth')

## MobileNet

In [ ]:
model_mobilenet = models.mobilenet_v2(pretrained=False)
model_mobilenet.classifier[1] = nn.Linear(model_mobilenet.last_channel, num_classes)
model_mobilenet = model_mobilenet.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mobilenet.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    start_time = time.time()
    model_mobilenet.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_mobilenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    end_time = time.time()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100.*correct/total:.2f}%, Time: {end_time - start_time:.2f}s')

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Loss: 0.6404, Accuracy: 64.70%, Time: 85.13s
Epoch [2/10], Loss: 0.5638, Accuracy: 73.46%, Time: 80.44s
Epoch [3/10], Loss: 0.5148, Accuracy: 74.92%, Time: 80.11s
Epoch [4/10], Loss: 0.4657, Accuracy: 79.61%, Time: 79.55s
Epoch [5/10], Loss: 0.4484, Accuracy: 80.10%, Time: 78.91s
Epoch [6/10], Loss: 0.4340, Accuracy: 81.16%, Time: 78.97s
Epoch [7/10], Loss: 0.4389, Accuracy: 81.26%, Time: 79.35s
Epoch [8/10], Loss: 0.4166, Accuracy: 81.79%, Time: 79.62s
Epoch [9/10], Loss: 0.3542, Accuracy: 84.31%, Time: 79.74s
Epoch [10/10], Loss: 0.3830, Accuracy: 84.31%, Time: 79.55s


In [ ]:
torch.save(model_mobilenet.state_dict(), 'model_mobilenet.pth')

##TINY CNN

In [ ]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(TinyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model_tinycnn = TinyCNN(num_classes=num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_tinycnn.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    start_time = time.time()
    model_tinycnn.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_tinycnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    end_time = time.time()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100.*correct/total:.2f}%')

Epoch [1/10], Loss: 1.0085, Accuracy: 64.17%
Epoch [2/10], Loss: 0.5210, Accuracy: 78.74%
Epoch [3/10], Loss: 0.4108, Accuracy: 84.50%
Epoch [4/10], Loss: 0.3195, Accuracy: 87.65%
Epoch [5/10], Loss: 0.2001, Accuracy: 93.06%
Epoch [6/10], Loss: 0.1273, Accuracy: 96.06%
Epoch [7/10], Loss: 0.1085, Accuracy: 96.36%
Epoch [8/10], Loss: 0.0579, Accuracy: 98.72%
Epoch [9/10], Loss: 0.0364, Accuracy: 99.26%
Epoch [10/10], Loss: 0.0298, Accuracy: 99.11%


In [ ]:
torch.save(model_tinycnn.state_dict(), 'model_tinycnn.pth')

#Quantization

In [ ]:
import torch
from torch.quantization import quantize_dynamic

def quantize_dynamic_int8(model, model_name):
    model.eval()
    quantized_model = quantize_dynamic(
        model, {torch.nn.Linear}, dtype=torch.qint8
    )
    torch.save(quantized_model.state_dict(), f"{model_name}_quantized_int8.pth")
    return quantized_model

In [ ]:
def evaluate_model(model, dataloader, device=torch.device("cpu")):
    model.to(device)
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy of Quantized Model: {accuracy:.2f}%")
    return accuracy


In [ ]:
!pip install --upgrade torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.quantization import prepare, convert, fuse_modules, get_default_qconfig

# Load your pretrained model
model_resnet18 = models.resnet18(pretrained=False)
model_resnet18.fc = nn.Linear(model_resnet18.fc.in_features, 2)
model_resnet18.load_state_dict(torch.load("resnet_solar_model.pth", map_location='cpu'))
model_resnet18.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def fuse_resnet18(model):
    # Fuse the layers following torchvision model structure
    for module_name, module in model.named_children():
        if module_name == "conv1":
            fuse_modules(model, ['conv1', 'bn1', 'relu'], inplace=True)
        elif module_name.startswith("layer"):
            for basic_block in module:
                fuse_modules(basic_block, ['conv1', 'bn1', 'relu'], inplace=True)
                fuse_modules(basic_block, ['conv2', 'bn2'], inplace=True)
    return model

model_resnet18 = fuse_resnet18(model_resnet18)


In [ ]:
model_resnet18.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model_resnet18, inplace=True)

/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


ResNet(
  (conv1): ConvReLU2d(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(inplace=True)
    (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
  )
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
      (bn1): Identity()
      (relu): Identity()
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

In [ ]:
def calibrate(model, dataloader):
    model.eval()
    with torch.no_grad():
        for i, (images, _) in enumerate(dataloader):
            if i >= 10: break
            model(images)

calibrate(model_resnet18, val_loader)


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [ ]:
torch.quantization.convert(model_resnet18, inplace=True)

ResNet(
  (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.03873179480433464, zero_point=0, padding=(3, 3))
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.040382321923971176, zero_point=0, padding=(1, 1))
      (bn1): Identity()
      (relu): Identity()
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.08442052453756332, zero_point=61, padding=(1, 1))
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.05300567299127579, zero_point=0, padding=(1, 1))
      (bn1): Identity()
      (relu): Identity()
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.11089125275611877, zero_point=65, padding=(1, 1))
    

In [ ]:
torch.save(model_resnet18.state_dict(), "resnet18_quantized.pth")

In [ ]:
def fuse_resnet10_modules(model):
    for m in model.modules():
        if isinstance(m, BasicBlock):
            torch.quantization.fuse_modules(m.residual, ['0', '1', '2'], inplace=True)
            torch.quantization.fuse_modules(m.residual, ['3', '4'], inplace=True)
            if len(m.shortcut) != 0:
                torch.quantization.fuse_modules(m.shortcut, ['0', '1'], inplace=True)
    torch.quantization.fuse_modules(model.conv1, ['0', '1', '2'], inplace=True)
    return model


In [ ]:
resnet10_q = ResNet10().to('cpu')
resnet10_q.eval()
resnet10_q.fuse_model = lambda: fuse_resnet10_modules(resnet10_q)
resnet10_q.fuse_model()

resnet10_q.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(resnet10_q, inplace=True)


ResNet10(
  (conv1): Sequential(
    (0): ConvReLU2d(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
    )
    (1): Identity()
    (2): Identity()
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (residual): Sequential(
        (0): ConvReLU2d(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (1): ReLU(inplace=True)
          (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
        )
        (1): Identity()
        (2): Identity()
        (3): Conv2d(
          32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
          (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
        )
        (4): Identity()
      )
      (shortcut): Sequential(
        (0): Conv2d(
          16, 32, kernel_size=(1, 1), stride=(2, 2)
          (activation_post_process): Hist

In [ ]:
calibrate(resnet10_q, val_loader)  # Few batches are enough

In [ ]:
torch.quantization.convert(resnet10_q, inplace=True)

ResNet10(
  (conv1): Sequential(
    (0): QuantizedConvReLU2d(3, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.00968098733574152, zero_point=0, padding=(1, 1))
    (1): Identity()
    (2): Identity()
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (residual): Sequential(
        (0): QuantizedConvReLU2d(16, 32, kernel_size=(3, 3), stride=(2, 2), scale=0.004081727471202612, zero_point=0, padding=(1, 1))
        (1): Identity()
        (2): Identity()
        (3): QuantizedConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), scale=0.003845191327854991, zero_point=63, padding=(1, 1))
        (4): Identity()
      )
      (shortcut): Sequential(
        (0): QuantizedConv2d(16, 32, kernel_size=(1, 1), stride=(2, 2), scale=0.008254588581621647, zero_point=63)
        (1): Identity()
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (residual): Sequential(
        (0): QuantizedConvReLU2d(32, 32, kernel_size=(3, 3), stride=(1, 1), scale=0.0016777031123638153, zer

In [ ]:
evaluate_model(resnet10_q, val_loader, device='cpu')

NotImplementedError: Could not run 'quantized::conv2d_relu.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d_relu.new' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMTIA, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered 

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the torchvision EfficientNet model
model_effnet = models.efficientnet_b0(pretrained=True)

# Modify the classifier for your dataset (assuming 2 classes here)
model_effnet.classifier[1] = nn.Linear(model_effnet.classifier[1].in_features, 2)
model_effnet.eval()  # Set to evaluation mode

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 73.8MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
from torch.quantization import quantize_dynamic

quantized_effnet = quantize_dynamic(
    model_effnet,
    {nn.Linear},  # Quantize only the Linear layers
    dtype=torch.qint8
)

In [ ]:
def evaluate_model(model, dataloader, device='cpu'):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f'Accuracy: {acc:.2f}%')
    return acc

# Run evaluation on CPU
evaluate_model(quantized_effnet, val_loader, device='cpu')

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 56.87%


56.866537717601545

In [ ]:
import torch
from torchvision import models
from torch.quantization import get_default_qconfig, prepare, convert
import copy

# Load pretrained model
mobilenet_v2 = models.mobilenet_v2(pretrained=True)
mobilenet_v2.eval()

# Fuse modules
fused_model = copy.deepcopy(mobilenet_v2)
fused_model.fuse_model()  # only works in recent torchvision versions

# Attach default static quantization config
fused_model.qconfig = get_default_qconfig('fbgemm')

# Prepare model for quantization
prepared_model = prepare(fused_model)

# Calibrate with representative data (run some batches through the model)
# Example with dummy data:
dummy_input = torch.randn(32, 3, 224, 224)
with torch.no_grad():
    prepared_model(dummy_input)

# Convert to quantized model
quantized_mobilenet = convert(prepared_model)

# Save the quantized model
torch.save(quantized_mobilenet.state_dict(), 'quantized_mobilenet_v2_static.pth')


In [ ]:
def evaluate_model(model, dataloader, device='cpu'):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f'Accuracy: {acc:.2f}%')
    return acc

# Run evaluation on CPU
evaluate_model(quantized_mobilenet, val_loader, device='cpu')

RuntimeError: apply_dynamic is not implemented for this packed parameter type